In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e22/sample_submission.csv')
train = pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e22/test.csv')
original_df = pd.read_csv('/kaggle/input/horse-survival-dataset/horse.csv')

dfs = pd.concat([train, original_df]).drop_duplicates()



print(dfs.shape)

ids, X, y = dfs['id'], dfs.drop(['id','outcome'], axis = 1), dfs['outcome']
dfs.head()

## Questions

 - is train and original model in similar 1d distribution and 2d mutual info?
 - is there well defined margins for interpolation problem?
 - can we extrapolate?
 - where do measurement error come from?
 - are errors iid? are errors with finite mean, var, and is there any stochasicity?


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def make_pairplot(df, target = 'outcome',
                 plot = False,
                 save = True,
                 path = "scatter.png"):



    # Separate the target variable (optional, adjust as needed)
    target = df[target]

    # Create a scatter matrix using Pandas
    scatter_matrix = pd.plotting.scatter_matrix(df, alpha=0.1, s=.1, c=pd.Categorical(target).codes, figsize=(8, 8), diagonal='hist')

    # Customize the plot labels, colors, and other settings
    for ax in scatter_matrix.ravel():
        ax.set_xlabel(ax.get_xlabel(), fontsize=12)
        ax.set_ylabel(ax.get_ylabel(), fontsize=12)
        ax.xaxis.label.set_rotation(45)
        ax.yaxis.label.set_rotation(0)
        ax.yaxis.labelpad = 20
        ax.grid(False)

    # Add a legend
    colors = ['blue', 'orange']
    labels = target.unique()
    legend_labels = {code: label for code, label in zip(pd.Categorical(target).codes, labels)}
    for code, label in legend_labels.items():
        scatter_matrix[0, 0].plot([], [], c=('red','blue','green','purple','orange')[code], label=label)

    # Show the legend
    scatter_matrix[0, 0].legend()

    # Save the scatter matrix to a file (e.g., PNG)
    plt.savefig(path)

    # Show the scatter matrix (optional)
    plt.show()


In [ ]:
# make_pairplot(train, path="train.png")
# make_pairplot(original_df, path="original.png")

In [ ]:
train.select_dtypes(include=['number']).columns

In [ ]:
original_df.columns

In [ ]:
###### df1 = train
df2 = original_df

# Function to create pair plots without Seaborn
def create_pair_plot_comparision(df, df2, title, target = "outcome"):
    num_features = df.select_dtypes(include=['number']).columns
    num_features_count = len(num_features)

    fig, axes = plt.subplots(nrows=num_features_count, ncols=num_features_count*2, figsize=(14, 7))
    fig.suptitle(title)

    for i, feature1 in enumerate(num_features):
        for j, feature2 in enumerate(num_features):
            if i == j:
                axes[i, j].hist(df[feature1], bins=10, alpha=0.7)
                axes[i, j].set_title(feature1)
            else:
                axes[i, j].scatter(df[feature2], df[feature1], c=df[target].astype('category').cat.codes, cmap='Set1', alpha=0.7)
                axes[i, j].set_xlabel(feature2)
                axes[i, j].set_ylabel(feature1)
    
    for i, feature1 in enumerate(num_features):
        for j, feature2 in enumerate(num_features):
            if i == j:
                axes[i,num_features_count+ j].hist(df2[feature1], bins=10, alpha=0.7)
                axes[i,num_features_count+ j].set_title(feature1)
            else:
                axes[i, num_features_count+ j].scatter(df2[feature2], df2[feature1], c=df2[target].astype('category').cat.codes, cmap='Set1', alpha=0.7)
                axes[i, num_features_count+ j].set_xlabel(feature2)
                axes[i, num_features_count+ j].set_ylabel(feature1)

    plt.tight_layout(rect=[0, 0, 2, 0.95])  # Adjust the layout
    plt.savefig('pairplots_combined.png')

    
create_pair_plot_comparision(train.drop('id', axis=1), original_df, "Comparing train df and original dataset")